___

<center><h1>Supply Chain Analysis</h1></center>

___

<center><h2>DSM020 - Data Programming in Python</h2></center><br>
<center><strong>Teacher:</strong> Sean McGrath </center>

___
<p></p>
<center style="color: #AA6373; font-weight: 400;"><strong>Presented by:</strong></center>
<center style="color: #AA6373; font-weight: 400;">Jorge Forero L.</center>
<center style="color: #AA6373; font-weight: 400;">Student Number: 240323983</center>
<center style="color: #AA6373; font-weight: 400;">Student Portal Username: JEFL1</center>
<center>August 2024</center>
<p></p>

In [1]:
# Common Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import s3fs

# Data Preparation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Modelling
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

# Testing and Evaluation
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve, accuracy_score
from sklearn.model_selection import GridSearchCV

## Data Understanding

This phase involves collecting the data, describing its attributes and performing the data exploratory analysis. For this coursework we use data that can be found in open data sources [1]. The data used to develop this project was selected given the requirements described by the instru ctor of the subject.

### Data Load

In [3]:
# Using the address for the .csv file from the GitHub repository
url = 'https://raw.githubusercontent.com/jforeroluque/Supply_Chain_Analysis/main/DataCoSupplyChainDataset.csv'

# Load the CSV file into a DataFrame, specifying the encoding
df_supply = pd.read_csv(url, encoding='ISO-8859-1', on_bad_lines='skip')

# Display the DataFrame
print(df_supply.head())

       Type  Days for shipping (real)  Days for shipment (scheduled)  \
0     DEBIT                         3                              4   
1  TRANSFER                         5                              4   
2      CASH                         4                              4   
3     DEBIT                         3                              4   
4   PAYMENT                         2                              4   

   Benefit per order  Sales per customer   Delivery Status  \
0          91.250000          314.640015  Advance shipping   
1        -249.089996          311.359985     Late delivery   
2        -247.779999          309.720001  Shipping on time   
3          22.860001          304.809998  Advance shipping   
4         134.210007          298.250000  Advance shipping   

   Late_delivery_risk  Category Id   Category Name Customer City  ...  \
0                   0           73  Sporting Goods        Caguas  ...   
1                   1           73  Sporting Goo

In [4]:
print(df_supply.columns)

Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Delivery Status',
       'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Email', 'Customer Fname', 'Customer Id',
       'Customer Lname', 'Customer Password', 'Customer Segment',
       'Customer State', 'Customer Street', 'Customer Zipcode',
       'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market',
       'Order City', 'Order Country', 'Order Customer Id',
       'order date (DateOrders)', 'Order Id', 'Order Item Cardprod Id',
       'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id',
       'Order Item Product Price', 'Order Item Profit Ratio',
       'Order Item Quantity', 'Sales', 'Order Item Total',
       'Order Profit Per Order', 'Order Region', 'Order State', 'Order Status',
       'Order Zipcode', 'Product Card Id', 'Product Category Id',
       'Product De

### Volume of the data

In [6]:
# Records
num_records = len(df_supply)

# Features
num_features = len(df_supply.columns)

print(f"The dataset has {num_records} records and {num_features} features.")

The dataset has 180519 records and 53 features.


### Features explanation 

|Feature| Explanation|
|:-:|:---|
|Type| Type of transaction made
|Days for shipping (real)| Actual shipping days of the purchased product
|Days for shipment (scheduled)| Days of scheduled delivery of the purchased product
|Benefit per order| Earnings per order placed
|Sales per customer| Total sales per customer made per customer
|Delivery Status| Delivery status of orders| Advance shipping , Late delivery , Shipping canceled , Shipping on tim...
|Late_delivery_risk| Categorical variable that indicates if sending is late (1), it is not late (0).
|Category Id| Product category code
|Category Name| Description of the product category
|Customer City| City where the customer made the purchase
|Customer Country| Country where the customer made the purchase
|Customer Email| Customer's email
|Customer Fname| Customer name
|Customer Id| Customer ID
|Customer Lname| Customer lastname
|Customer Password| Masked customer key
|Customer Segment| Types of Customers| Consumer , Corporate , Home Office
|Customer State| State to which the store where the purchase is registered belongs
|Customer Street| Street to which the store where the purchase is registered belongs
|Customer Zipcode| Customer Zipcode
|Department Id| Department code of store
|Department Name| Department name of store
|Latitude| Latitude corresponding to location of store
|Longitude| Longitude corresponding to location of store
|Market| Market to where the order is delivered | Africa , Europe , LATAM , Pacific Asia , USCA
|Order City| Destination city of the order
|Order Country| Destination country of the order
|Order Customer Id| Customer order code
|order date (DateOrders)| Date on which the order is made
|Order Id| Order code
|Order Item Cardprod Id| Product code generated through the RFID reader
|Order Item Discount| Order item discount value
|Order Item Discount Rate| Order item discount percentage
|Order Item Id| Order item code
|Order Item Product Price| Price of products without discount
|Order Item Profit Ratio| Order Item Profit Ratio
|Order Item Quantity| Number of products per order
|Sales| Value in sales
|Order Item Total| Total amount per order
|Order Profit Per Order| Order Profit Per Order
|Order Region| Region of the world where the order is delivered | Southeast Asia ,South Asia ,Oceania ,Eastern ...
|Order State| State of the region where the order is delivered
|Order Status| Order Status | COMPLETE , PENDING , CLOSED , PENDING_PAYMENT ,CANCELED , PROCESSING ,SUSPECTED_FR...
|Product Card Id| Product code
|Product Category Id| Product category code
|Product Description| Product Description
|Product Image| Link of visit and purchase of the product
|Product Name| Product Name
|Product Price| Product Price
|Product Status| Status of the product stock |If it is 1 not available , 0 the product is available
|Shipping date (DateOrders)| Exact date and time of shipment
|Shipping Mode| The following shipping modes are presented | Standard Class , First Class , Second Class , Same D..."

## Exploratory Data Analysis

### Missing Values

We will be dropping the rows with missing values in the features that are the most relevant for the analysis we will be doing in this project. 

In [7]:
# Dropping rows with missing values in critical columns
df_cleaned = df_supply.dropna(subset=[
    "Late_delivery_risk",
    "Order Item Product Price",
    "Order Item Quantity",
    "Days for shipment (scheduled)",
    "Days for shipping (real)",
    "Customer Segment",
    "Order Country",
    "Shipping Mode"
])

# Show the first 5 rows of cleaned data
print(df_cleaned.head(5))

       Type  Days for shipping (real)  Days for shipment (scheduled)  \
0     DEBIT                         3                              4   
1  TRANSFER                         5                              4   
2      CASH                         4                              4   
3     DEBIT                         3                              4   
4   PAYMENT                         2                              4   

   Benefit per order  Sales per customer   Delivery Status  \
0          91.250000          314.640015  Advance shipping   
1        -249.089996          311.359985     Late delivery   
2        -247.779999          309.720001  Shipping on time   
3          22.860001          304.809998  Advance shipping   
4         134.210007          298.250000  Advance shipping   

   Late_delivery_risk  Category Id   Category Name Customer City  ...  \
0                   0           73  Sporting Goods        Caguas  ...   
1                   1           73  Sporting Goo